In [ ]:
import pandas as pd

data = pd.read_json("https://raw.githubusercontent.com/krisrs1128/distortions-data/main/data/mammoth_3d.json")
data.columns = ["x", "y", "z"]
data = data[:500]

In [ ]:
from anndata import AnnData
import scanpy as sc

n_neighbors = 50
adata = AnnData(X=data, obs=pd.DataFrame(index=data.index))
sc.pp.neighbors(adata, n_neighbors=n_neighbors)
sc.tl.umap(adata)
embedding = adata.obsm["X_umap"].copy()

In [ ]:
import numpy as np

radius = 3
parameters = {
    "radius": 2 ** (np.linspace(-1, 1, 10)) * radius,
    "scaling_epps": np.linspace(3, 7, 10)
}

In [ ]:
from distortions.geometry import Geometry
from distortions.geometry import expand_geoms
from distortions.geometry import metric_sensitivity

n_neighbors = 50
geom = Geometry("brute", laplacian_method="geometric", affinity_kwds={"radius": radius}, adjacency_kwds={"n_neighbors": n_neighbors}, laplacian_kwds={"scaling_epps": 5})
geoms, params_df = expand_geoms(geom, parameters)
Hvv, Hs = metric_sensitivity(geoms, embedding, data)

In [ ]:
plot_data = []
for i in range(Hs.shape[1]):
    for j in range(Hs.shape[0]):
        plot_data.append({
            "sample": i,
            "combination": j,
            "hv00": float(Hvv[j, i, 0, 0]),
            "hv11": float(Hvv[j, i, 1, 1]),
            "hv01": float(Hvv[j, i, 0, 1]),
            "hs0": float(Hs[j, i, 0]),
            "hs1": float(Hs[j, i, 1])
        })

params_df["combination"] = params_df.index
plot_df = pd.DataFrame(plot_data)
plot_df = plot_df.merge(params_df)

In [ ]:
import altair as alt
alt.data_transformers.enable("vegafusion")

random_samples = np.random.choice(plot_df["sample"].unique(), size=20, replace=False)

charts = []
for i in range(len(random_samples)):
    random_subset = plot_df[plot_df["sample"] == i]
    chart_ = alt.Chart(random_subset).mark_rect().encode(
        x=alt.X('radius:N', title='Radius', axis=alt.Axis(format='.2f')),
        y=alt.Y('scaling_epps:N', title='Epsilon', axis=alt.Axis(format='.2f')),
        fill=alt.Fill('hv00:Q', title='h')
    )
    charts.append(chart_)

[display(p) for p in charts]

In [ ]:
charts = []
for i in range(len(random_samples)):
    random_subset = plot_df[plot_df["sample"] == i]
    chart_ = alt.Chart(random_subset).mark_rect().encode(
        x=alt.X('radius:N', title='Radius', axis=alt.Axis(format='.2f')),
        y=alt.Y('scaling_epps:N', title='Epsilon', axis=alt.Axis(format='.2f')),
        fill=alt.Fill('hs0:Q', title='h')
    )
    charts.append(chart_)

[display(p) for p in charts]

In [ ]:
import altair as alt
alt.data_transformers.enable("vegafusion")

alt.Chart(plot_df).mark_line(size=0.2).encode(
    x=alt.X('scaling_epps:Q', title='Rescaling Epsilon'),
    y=alt.Y('hv00:Q', title='h'),
    detail=['sample:N', 'radius:Q']
)

In [ ]:
alt.Chart(plot_df).mark_line(size=0.2).encode(
    x=alt.X('radius:Q', title='Radius'),
    y=alt.Y('hv00:Q', title='h'),
    detail=['sample:N', 'scaling_epps:Q']
)

In [ ]:
alt.Chart(plot_df).mark_line(size=0.2).encode(
    x=alt.X('scaling_epps:Q', title='Rescaling Epsilon'),
    y=alt.Y('hs0:Q', title='h'),
    detail=['sample:N', "radius:Q"]
)

In [ ]:
alt.Chart(plot_df).mark_line(size=0.2).encode(
    x=alt.X('radius:Q', title='Laplacian Radius'),
    y=alt.Y('hs0:Q', title='h'),
    detail=['sample:N', "scaling_epps:Q"]
)